## Imports

In [1]:
!pip install sentence_transformers~=2.2.2
!pip install chromadb
!pip install deepeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=87abc24b8f96b74d1a03fae6ab3e78b4eea5378bd4190d08008373827e0f3e07
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 604.0/604.0 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 

In [3]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('openai')

In [45]:
from itertools import zip_longest
import re
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCase
from deepeval.test_case import LLMTestCaseParams
from deepeval.dataset import EvaluationDataset

from dotenv import find_dotenv, load_dotenv

_ = load_dotenv(find_dotenv())

# TODO:
# Что то придумать с сохранением результатов оценки

OPENAI_MODEL = 'gpt-4o-mini'

def create_deepeval_dataset(
    user_queries: list[str],
    actual_outputs: list[str],
    expected_outputs: list[str] | None = None,
    expected_contexts: list[list[str]] | None = None,
    actual_contexts: list[list[str]] | None = None
) -> EvaluationDataset:

    # Проверка длины обязательных списков
    if len(actual_outputs) != len(user_queries):
        raise ValueError(f"Length mismatch: 'actual_outputs' has {len(actual_outputs)} items, "
                         f"but 'user_queries' has {len(user_queries)}.")

    # Проверка длины необязательных списков
    if expected_outputs and len(expected_outputs) != len(user_queries):
        raise ValueError(f"Length mismatch: 'expected_outputs' has {len(expected_outputs)} items, "
                         f"but 'user_queries' has {len(user_queries)}.")

    if expected_contexts and len(expected_contexts) != len(user_queries):
        raise ValueError(f"Length mismatch: 'expected_contexts' has {len(expected_contexts)} items, "
                         f"but 'user_queries' has {len(user_queries)}.")

    if actual_contexts and len(actual_contexts) != len(user_queries):
        raise ValueError(f"Length mismatch: 'actual_contexts' has {len(actual_contexts)} items, "
                         f"but 'user_queries' has {len(user_queries)}.")

    # Создание тест-кейсов
    test_cases = []

    for query, actual_output, expected_output, expected_context, actual_context in zip_longest(
        user_queries,
        actual_outputs,
        expected_outputs or [],
        expected_contexts or [],
        actual_contexts or []
    ):
        test_case_kwargs = {
            'input': query,
            'actual_output': actual_output,
            **({'expected_output': expected_output} if expected_output else {}),
            **({'context': expected_context} if expected_context else {}),
            **({'retrieval_context': actual_context} if actual_context else {})
        }

        test_cases.append(LLMTestCase(**test_case_kwargs))

    return EvaluationDataset(test_cases=test_cases)

# NOTE: Метрики
# 1. Документы релевантны вопросу?
# 2. Отвечает ли ответ на вопрос?
# 3. Галлюцинации: ответ основан на найденных документах?

relevance_metric = GEval(
    model=OPENAI_MODEL,
    name="Document Relevance",
    strict_mode=True,  # Enforce strict binary scoring
    evaluation_steps=[
        "Check if the 'retrieval_context' contains documents that are directly relevant to the 'input'. Penalize if the documents are not related to the user's question.",
        "Verify that key terms or concepts from the 'input' are represented in the 'retrieval_context'. Heavily penalize if any critical information is missing.",
        "Ensure that the 'retrieval_context' covers all necessary facts or details to fully answer the 'input'. Penalize for omissions of important information.",
        "Allow for minor irrelevant content in the 'retrieval_context', but penalize if it significantly distracts from the relevance to the 'input'."
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT,  # The user's question or query
        LLMTestCaseParams.RETRIEVAL_CONTEXT  # The retrieved documents to check for relevance
    ]
)

answer_relevance_metric = GEval(
    model=OPENAI_MODEL,
    name="Answer Relevance and Completeness",
    strict_mode=True,  # Enforce strict binary scoring
    evaluation_steps=[
        "Check if the 'actual_output' directly addresses the main point(s) of the 'input'. Penalize if the answer is off-topic or misses the core question.",
        "Verify whether the facts in the 'actual_output' align with the 'input' and any provided context. Penalize if any factual contradictions are present.",
        "Heavily penalize omission of critical details from the 'actual_output'. Ensure that all relevant parts of the 'input' are addressed.",
        "Minor vagueness or unnecessary information can be tolerated, but penalize if it confuses the user or detracts from the clarity of the response."
    ],
    evaluation_params=[
        LLMTestCaseParams.INPUT,  # The user's question or query
        LLMTestCaseParams.ACTUAL_OUTPUT  # The generated answer to evaluate
    ]
)

hallucination_detection_metric = GEval(
    model=OPENAI_MODEL,
    name="Hallucination Detection",
    strict_mode=True,
    evaluation_steps=[
        "Check if all factual information in the 'actual_output' is grounded in the 'retrieval_context'. Penalize if any information is introduced that is not supported by the retrieved documents.",
        "Heavily penalize if key facts in the 'actual_output' are fabricated or not present in the 'retrieval_context'.",
        "Ensure that the 'actual_output' accurately reflects the information in the 'retrieval_context'. Slight vagueness or subjectivity is allowed if it does not introduce new or false facts.",
        "Minor omissions from the 'retrieval_context' can be tolerated, but the answer should not rely on unsupported or external information."
    ],
    evaluation_params=[
        LLMTestCaseParams.ACTUAL_OUTPUT,  # The model's generated answer
        LLMTestCaseParams.RETRIEVAL_CONTEXT  # The documents used to generate the answer
    ]
)

correctness_metric = GEval(
    model=OPENAI_MODEL,
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the expected output.",
    # NOTE: you can only provide either criteria or evaluation_steps, and not both
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'expected output'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
)

def g_evaluate(
    user_queries: list[str],
    actual_outputs: list[str],
    expected_outputs: list[str] | None = None,
    expected_contexts: list[list[str]] | None = None,
    actual_contexts: list[list[str]] | None = None
) -> EvaluationDataset:
    dataset = create_deepeval_dataset(
        user_queries=user_queries,
        actual_outputs=actual_outputs,
        expected_outputs=expected_outputs,
        actual_contexts=retrieval_contexts
    )

    # Evaluate the dataset using the defined metrics
    dataset.evaluate([relevance_metric, answer_relevance_metric, hallucination_detection_metric, correctness_metric])

    return dataset


In [5]:
import math


def calculate_dcg(fractions):
    """
    Вычисляет Discounted Cumulative Gain (DCG) для списка значений fraction.
    """
    dcg = 0.0
    for i, rel in enumerate(fractions):
        dcg += rel / math.log2(i + 2)  # Позиции начинаются с 1, поэтому i+2
    return dcg


def calculate_ndcg(retrieved_fractions):
    """
    Вычисляет NDCG для каждого списка retrieved_fractions.

    Args:
        retrieved_fractions (list): Список списков значений fraction для каждого retrieved_chunk.

    Returns:
        list: Список значений NDCG для каждого списка.
    """
    ndcg_values = []

    for fractions in retrieved_fractions:
        # Рассчитываем DCG
        dcg = calculate_dcg(fractions)

        # Рассчитываем идеальный DCG (IDCG), сортируя fractions по убыванию
        ideal_fractions = sorted(fractions, reverse=True)
        idcg = calculate_dcg(ideal_fractions)

        # Рассчитываем NDCG
        if idcg == 0:
            ndcg_values.append(0.0)
        else:
            ndcg_values.append(dcg / idcg)

    return ndcg_values



In [6]:
def calculate_recall(ground_truth_lists, float_lists):
    """
    Рассчитывает значение recall для каждого списка ground_truth.

    Args:
        ground_truth_lists (list): Список списков со строками ground_truth.
        float_lists (list): Список списков с флоатами, которые соответствуют значению метрик для каждого ground_truth.

    Returns:
        list: Список значений recall (float от 0 до 1) для каждого блока данных.
    """
    recall_values = []

    # Проходим по каждому списку ground_truth и соответствующему списку с float
    for ground_truth, float_list in zip(ground_truth_lists, float_lists):
        num_ground_truth = len(ground_truth)  # Количество строк в ground_truth

        if num_ground_truth == 0:
            recall_values.append(0.0)  # Если ground_truth пуст, recall = 0
        else:
            total_floats = sum(float_list)  # Суммируем все значения float
            recall = total_floats / num_ground_truth  # Делим сумму флоатов на количество строк ground_truth
            recall_values.append(min(recall, 1.0))  # Убеждаемся, что recall не превышает 1

    return recall_values



In [7]:
import requests

import pandas as pd
from tqdm import tqdm


from sentence_transformers import SentenceTransformer

import chromadb

# # Только для юпитер-ноутбука
# import sys
# import os
# sys.path.append(os.path.abspath('../'))
# # -----

# from src.metrics.geval import LLMTestCase, relevance_metric, answer_relevance_metric, hallucination_detection_metric
# from src.metrics.recall import calculate_recall
# from src.metrics.ndcg import calculate_ndcg
tqdm.pandas()

In [8]:
DEVICE = 'cuda'
# CHROMA_DB_PATH = '../data/chroma_db'
# DATA_FOR_DB_PARQUET = '../data/actual/interim/data_for_db_corrected.parquet'
# EVALUATION_PARQUET = '../data/actual/interim/evaluation_df.parquet'
# SMALL_EVALUATION_PARQUET = '../data/actual/interim/evaluation_df_125.parquet'

CHROMA_DB_PATH = 'chroma_db'
DATA_FOR_DB_PARQUET = '/content/data_for_db_corrected.parquet'
EVALUATION_PARQUET = '/content/evaluation_df.parquet'
SMALL_EVALUATION_PARQUET = '/content/evaluation_df_125.parquet'

## Data for db

In [9]:
data_for_db = pd.read_parquet(DATA_FOR_DB_PARQUET)

In [10]:
data_for_db.head(1)

,index,kb_query,kb_answer,class_1,class_2,kb_query_orig
0,0,Что нельзя публиковать на RUTUBE (рутуб)?,Чужой контент без разрешения автора или правоо...,МОДЕРАЦИЯ,Отклонение/блокировка видео,Что нельзя публиковать на RUTUBE?


In [11]:
data_for_db = data_for_db.fillna('')

## Embedder

In [12]:
embedder = SentenceTransformer('intfloat/multilingual-e5-large',device=DEVICE)

# Обратите внимание на prompt
#  query:
#  passage:

# input_texts = [
#     'query: how much protein should a female eat',
#     'query: 南瓜的家常做法',
#     "passage: As a general guideline, the CDC's average requirement of protein for women ages 19 to 70 i     s 46 grams per day. But, as you can see from this chart, you'll need to increase that if you're expecting or traini     ng for a marathon. Check out the chart below to see how much protein you should be eating each day.",
#     "passage: 1.清炒南瓜丝 原料:嫩南瓜半个 调料:葱、盐、白糖、鸡精 做法: 1、南瓜用刀薄薄的削去表面一层皮     ,用勺子刮去瓤 2、擦成细丝(没有擦菜板就用刀慢慢切成细丝) 3、锅烧热放油,入葱花煸出香味 4、入南瓜丝快速翻炒一分钟左右,     放盐、一点白糖和鸡精调味出锅 2.香葱炒南瓜 原料:南瓜1只 调料:香葱、蒜末、橄榄油、盐 做法: 1、将南瓜去皮,切成片 2、油     锅8成热后,将蒜末放入爆香 3、爆香后,将南瓜片放入,翻炒 4、在翻炒的同时,可以不时地往锅里加水,但不要太多 5、放入盐,炒匀      6、南瓜差不多软和绵了之后,就可以关火 7、撒入香葱,即可出锅"
# ]
# embeddings = embedder.encode(input_texts, normalize_embeddings=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_hub_download` instead.
  warnings.warn(warning_message, FutureWarning)


.gitattributes:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/201 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/160k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

Here are some rules of thumb:

Use "query: " and "passage: " correspondingly for asymmetric tasks such as passage retrieval in open QA, ad-hoc information retrieval.

Use "query: " prefix for symmetric tasks such as semantic similarity, bitext mining, paraphrase retrieval.

Use "query: " prefix if you want to use embeddings as features, such as linear probing classification, clustering.

## Database

In [13]:
client = chromadb.PersistentClient(path=CHROMA_DB_PATH, settings=chromadb.Settings(allow_reset=True))

In [14]:
client.reset()

True

In [15]:
collection = client.get_or_create_collection(name="rutube",metadata={"hnsw:space": "cosine"})

In [16]:
documents = data_for_db['kb_query'].tolist()

In [17]:
documents = ['query: ' + doc for doc in documents]
embeddings = embedder.encode(documents, show_progress_bar=True)

Batches:   0%|          | 0/9 [00:00<?, ?it/s]

In [18]:
metadatas = data_for_db.drop(columns=['index','kb_query']).to_dict(orient='index')
metadatas = [data[1] for n,data in enumerate(metadatas.items())]

In [19]:
ids = [str(x) for x in data_for_db['index'].tolist()]

In [20]:
collection.add(
    documents=documents,
    embeddings=embeddings,
    metadatas=metadatas,
    ids=ids
)

## Retriever

In [21]:
def retrieve_relevant_chunks(query:str | list[str], where: dict | None = None, where_document: dict | None = None, n_results:int = 3) -> None:
    '''
    where={"metadata_field": "is_equal_to_this"},
    where_document={"$contains":"search_string"}
    '''

    if isinstance(query,str):
        query=[query]

    query = ['query: ' + q for q in query]

    query_embeddings = embedder.encode(query)

    query_kwargs = {
        'query_embeddings': query_embeddings,
        'include': ['documents','metadatas'],
        'n_results': n_results
    }
    if where:
        query_kwargs['where']=where

    if where_document:
        query_kwargs['where_document'] = where_document

    result = collection.query(**query_kwargs)
    return result

def format_retrieve_result(retrieve_result) -> list:
    result = []

    ids = retrieve_result.get('ids', [])
    documents = retrieve_result.get('documents', [])
    metadatas = retrieve_result.get('metadatas', [])

    for i in range(len(ids)):
        group = []
        for j in range(len(ids[i])):
            entry = {
                'id': ids[i][j],
                'document': documents[i][j],
                'metadata': metadatas[i][j] if metadatas else None
            }
            group.append(entry)
        result.append(group)

    return result

def extract_context_for_generation(retrieved_result: list[dict], include_question: bool = True) -> str:
    result_string = "\n\n".join(
        [
            (f"Вопрос: {item['document']}\nОтвет: {item['metadata']['kb_answer']}" if include_question else item['metadata']['kb_answer'])
            for sublist in retrieved_result for item in sublist
        ]
    )

    return result_string

## Generation

In [22]:
def generate(prompt:str, temperature:float=0, return_str_only:bool=True) -> dict:
    result = requests.post(
        "http://89.169.135.235:11434/api/generate",
        json={
            "model": "mistral-nemo",
            "options": {"seed": 123, "temperature": temperature},
            "prompt": prompt,
            "stream": False,
        },
    ).json()
    if return_str_only:
        return result['response']
    else:
        return result

## Chunk LLM filtering prompt (optional)

In [23]:
FILTERING_PROMPT = '''
'''.strip()

In [24]:
def chunk_filtering(input:str,query:str) -> bool:
    pass

## Query rewriting

In [54]:
QUERY_REWRITING_PROMPT_TEMPLATE = '''
Ты интеллектуальный помощник службы поддержки RUTUBE.
Тебе будет дан ВОПРОС.
Твоя задача - переписать вопрос без изменения смысла, но в нейтральном тоне, в котором вопросы сохранялись бы в базу знаний.
Если вопрос и так в нейтральном тоне, не переписывай. Не искажай исходный вопрос.

ВОПРОС:
=====
{query}
=====

=====
'''.strip()

In [55]:
def rewrite_query(user_input):
  response = generate(QUERY_REWRITING_PROMPT_TEMPLATE.format(query=user_input))
  return response

In [71]:
list_of_queries = ['Почему могут отключить монетизацию из-за авторского права на видео и как этого избежать?',
       'Какой контент можно использовать для монетизации, то есть в коммерческих целях?',
       'Кому принадлежат авторские права на контент, который я размещаю на платформе?',
       'Можно ли цитировать чужие произведения?',
       'Нарушает ли авторские права использование образа персонажа игры/фильма/сериала?',
       'Что такое мобильное приложение Студия RUTUBE?',
       'Какие функции доступны в приложении Студия RUTUBE?',
       'На каких операционных системах доступно приложение Студия RUTUBE?',
       'Как узнать версию приложения Студия RUTUBE?',
       'Как зарегистрироваться в приложении Студия RUTUBE?',
       'Как авторизоваться в приложении Студия RUTUBE?',
       'Можно ли авторизоваться в приложении Студия RUTUBE под уже существующим аккаунтом?',
       'Можно ли авторизоваться (выполнить вход) в приложении Студия RUTUBE без ввода проверочного кода?',
       'Как выйти из аккаунта в приложении Студия RUTUBE?',
       'Можно ли изменять данные профиля в приложении Студия RUTUBE?',
       'Можно ли изменять название и описание профиля канала в приложении Студия RUTUBE?',
       'Можно ли удалить канал через приложение Студия RUTUBE?',
       'Можно ли изменить обложку канала и фото профиля в приложении Студия RUTUBE?',
       'Можно ли выдать доступы к студии канала через приложение Студия RUTUBE?',
       'Можно ли управлять другим каналом через приложение Студия RUTUBE?',
       'Какие требования к обложке канала?',
       'Можно ли менять местами или скрыть разделы во вкладке "Обзор" в приложении Студия RUTUBE?',
       'Как посмотреть аналитику по конкретному видео в приложении Студия RUTUBE?',
       'Как посмотреть доход по конкретному видео в приложении Студия RUTUBE?',
       'Как подключиться к партнерской программе в приложении Студия RUTUBE?',
       'Как создать плейлисты в приложении Студия RUTUBE?',
       'Можно ли в приложении Студия RUTUBE создавать и вести трансляции?',
       'Какие требования к обложке видео?',
       'Можно ли работать с комментариями в приложении Студия RUTUBE?',
       'Как написать обращение в поддержку через приложение Студия RUTUBE? ',
       'Где можно скачать приложение Студия RUTUBE?',
       'Доступно ли приложение Студия RUTUBE на планшетах?',
       'Можно ли заблокировать пользователя, оставляющего оскорбительные комментарии?',
       'Сколько комментариев можно закрепить под видео?',
       'Как перенести видео с YouTube на RUTUBE?',
       'К какому количеству студий может быть одновременно предоставлен доступ одному пользователю?',
       'Может ли один пользователь занимать сразу несколько ролей доступа в рамках студии одного канала?',
       'Какие бывают роли в доступах к студии канала?',
       'Сколько пользователей можно назначить на роль Редактора?',
       'Какие возможности взаимодействия со Студией доступны пользователю в роли Редактора с ограниченными правами?',
       'Сколько пользователей можно назначить на роль Редактора с ограниченными правами?',
       'Какие возможности взаимодействия со Студией доступны пользователю с правами Модератора комментариев?',
       'Каким ролям в студии доступен функционал работы с комментариями?',
       'Что такое Shorts на RUTUBE?',
       'Как загрузить видео в формате Shorts?',
       'Где расположен раздел Shorts?',
       'Как видео попадают в раздел Shorts на главной?',
       'Как увеличить шансы на попадание моего видео в раздел Shorts на главной странице?',
       'Можно ли включить ролик Shorts в горизонтальном плеере?',
       'Что такое Smart TV?',
       'Где можно скачать приложение RUTUBE для Smart TV?',
       'Как установить RUTUBE на телевизоры LG, Samsung, Hisense, если приложения RUTUBE нет во встроенном магазине?',
       'Будет ли приложение работать без обновлений на старых моделях Smart TV?',
       'Сколько телевизоров или ТВ-приставок может быть привязано к одному профилю RUTUBE?',
       'Как войти в профиль, если при сканировании QR-кода не открывается страница входа?',
       'Какие горячие клавиши есть на RUTUBE?',
       'Что такое трансляции (стримы) на RUTUBE?',
       'В каких приложениях доступны прямые трансляции?',
       'Кому доступно проведение трансляций (стримов) на RUTUBE?',
       'Можно ли запускать несколько трансляций одновременно?',
       'Есть ли реклама на трансляциях?',
       'Могут ли трансляции монетизироваться?',
       'Что я могу и что не могу делать во время трансляции (стрима)?',
       'Что такое приватная трансляция на RUTUBE?',
       'Можно ли эмбедировать (встраивать) трансляции на сторонние сайты?',
       'Можно ли эмбедировать (встраивать) приватные трансляции на сторонние сайты?',
       'Можно ли вести трансляции на RUTUBE, если подать поток в 60 FPS?',
       'Если в трансляцию подать поток 60 FPS, то сохранится ли запись трансляции с этой кадровой частотой?',
       'Можно ли выпивать на стримах (в пределах разумного) и можно ли вейпить?',
       'Как присвоить трансляции (стриму) категорию 18+?',
       'Как продвигаются стримы?', 'Как создать трансляцию?',
       'Как запустить приватную трансляцию?',
       'Сколько может длиться трансляция (стрима)?',
       'Можно ли открыть доступ к просмотру записи приватной трансляции для всех зрителей?',
       'Как настроить vMix для проведения трансляций на RUTUBE? ',
       'Что такое встраивание (embed) видео?',
       'Какие настройки параметров можно сделать в коде вставки',
       'Как мне лучше оформить свой канал?',
       'Какие рекомендации следует учитывать при создании обложки и заголовка для видео, чтобы повысить его привлекательность и видимость? ',
       'Какие пользователи могут подать на верификацию канала?',
       'Как подать заявку на верификацию канала?',
       'Может ли физическое лицо получить статус?',
       'Как поменять URL своего RUTUBE канала?',
       'Обязательна ли верификация канала для подключения к монетизации?',
       'Что делать, если заявку не одобрили?',
       'Моя заявка находится длительное время без ответа. Что делать?',
       'Что такое Монетизация на RUTUBE?',
       'С какого возраста доступна монетизация?',
       'Могу ли я подключиться к партнёрской программе, если я не резидент РФ?',
       'Как подключить монетизацию, если я физлицо?',
       'Какие документы нужны для заявки на монетизацию?',
       'Монетизация подключена. С какого момента будет рассчитываться вознаграждение? ',
       'Сколько времени дается на исправление нарушений на канале?',
       'Как долго рассматривается повторная заявка на монетизацию?',
       'Где посмотреть подробную аналитику по монетизации моего канала и информацию о начислениях?',
       'Как формируется вознаграждение за монетизацию?',
       'Как мне вывести средства с баланса в личном кабинете?',
       'Куда я могу отправить уведомление об исключительности прав?',
       'Кем осуществляется модерация в RUTUBE?',
       'Что значит «Ваше видео отклонено»?',
       'Забыл свой пароль, а письмо для его восстановления не приходит на почту',
       'Что делать, если у меня уже нет доступа к ранее указанному адресу электронной почты?',
       'Как изменить привязанный к аккаунту номер телефона?',
       'Пробую авторизоваться, а сервис пишет, что такого пользователя не существует или пароль неверный. Что делать?',
       'Я хочу удалить свой аккаунт. Как это сделать?',
       'Что такое личный канал на RUTUBE?',
       'Как завести канал на RUTUBE?',
       'Как изменить название моего канала на RUTUBE?',
       'Как мне получать донаты за публикации?',
       'Как включить функцию «Поддержать автора»?',
       'На какой сервис можно добавить ссылку?',
       'Как получить ссылку в этих сервисах?',
       'Кто такой правообладатель и как им стать?',
       'Как проходит подтверждение номера телефона?',
       'Что такое онлайн-чат и как им управлять?',
       'Что произойдет с моей трансляцией после ее окончания?',
       'Сохраняется ли приватная трансляция?',
       'Могу ли я скачать копию моей трансляции?',
       'Какие технические требования предъявляются к загружаемым видеороликам?',
       'Не могу посмотреть ролик в хорошем качестве',
       'Какую категорию выбрать для видео?',
       'Какие субтитры используются в RUTUBE?',
       'Правила размещения субтитров']

In [75]:
user_input = list_of_queries[3]
print(user_input)
rewrite_query(user_input)

Можно ли цитировать чужие произведения?


'Можно ли цитировать чужие произведения?'

## Prompt template

In [25]:
PROMPT_TEMPLATE = '''
Ты интеллектуальный помощник службы поддержки RUTUBE.
RUTUBE — ведущий российский видеопортал, предлагающий к просмотру тв онлайн, кинофильмы, сериалы, мультфильмы и пользовательское видео.
Ты очень вежливый и дружелюбный. Твоя задача ответить на ВОПРОС используя КОНТЕКСТ.

У тебя есть только знания из контекста и этого промпта.
Если ответа на вопрос к контексте нет, то напиши, что не знаешь ответа.
Отвечай только используя контекст, не используй другие знания.

ВОПРОС:
=====
{query}
=====

КОНТЕКСТ:
=====
{context}
=====
'''.strip()

## Complete QA-RAG pipeline

In [76]:
def rag_pipeline(input:str) -> dict:
    rewrited_input = rewrite_query(input)
    actual_context_raw = retrieve_relevant_chunks(rewrited_input,n_results=5)
    actual_context = format_retrieve_result(actual_context_raw)
    actual_context_to_llm = extract_context_for_generation(actual_context)
    prompt = PROMPT_TEMPLATE.format(query=input,context=actual_context_to_llm)
    response = generate(prompt)


    result = {
        'input':input,
        'actual_context': actual_context,
        'actual_response': response
    }

    return result

rag_pipeline('Как включить монетизацию?')

{'input': 'Как включить монетизацию?',
 'actual_context': [[{'id': '173',
    'document': 'query: Как подключить монетизацию, если я физлицо?',
    'metadata': {'class_1': 'ПРЕДЛОЖЕНИЯ',
     'class_2': 'Монетизация',
     'kb_answer': 'Увы, физлицам монетизация недоступна, зато доступна самозанятым. Стать самозанятым можно за пару минут через приложение «Мой налог».  ',
     'kb_query_orig': 'Как подключить монетизацию, если я физлицо?'}},
   {'id': '189',
    'document': 'query: Как формируется вознаграждение за монетизацию?',
    'metadata': {'class_1': 'МОНЕТИЗАЦИЯ',
     'class_2': 'Статистика по монетизации',
     'kb_answer': 'Для расчета вознаграждения партнера используется специальная формула, указанная в п.3.1. Оферты. Актуальные показатели формирующие общее итоговое вознаграждение партнера также отображается в Студии RUTUBE (студии рутуб) в разделе «Монетизация»: https://studio.rutube.ru/monetization',
     'kb_query_orig': 'Как формируется вознаграждение за монетизацию?'}},

## Evaluation

In [77]:
eval_df = pd.read_parquet(EVALUATION_PARQUET)

In [78]:
eval_df.head(2)

,input,expected_output,kb_query,index,kb_query_orig
0,Здравствуйте! Можно уточнить причины Правилhtt...,Добрый день!\nЧто нельзя публиковать на RUTUBE...,Что нельзя публиковать на RUTUBE?,0.0,Что нельзя публиковать на RUTUBE?
1,"Добрый вечер, какой топ причин блокировки виде...",Добрый вечер!\nЧто заперщено публиковать на RU...,Что нельзя публиковать на RUTUBE?,0.0,Что нельзя публиковать на RUTUBE?


In [79]:
# def stratified_sample(df, stratify_column, frac):
#     return df.groupby(stratify_column, group_keys=False).apply(lambda x: x.sample(frac=frac))

# stratified_df = stratified_sample(eval_df, stratify_column='index', frac=0.2)


In [80]:
stratified_df = pd.read_parquet(SMALL_EVALUATION_PARQUET)

In [81]:
def process_row(row):
    result = rag_pipeline(row['input'])

    actual_response = result.get('actual_response', '')

    actual_context_ids = [item['id'] for sublist in result['actual_context'] for item in sublist]

    class_1_list = [item['metadata']['class_1'] for sublist in result['actual_context'] for item in sublist]
    class_2_list = [item['metadata']['class_2'] for sublist in result['actual_context'] for item in sublist]

    actual_context = extract_context_for_generation(result['actual_context'])

    return pd.Series({
        'actual_response': actual_response,
        'actual_context_ids': actual_context_ids,
        'actual_context': actual_context,
        'class_1': class_1_list,
        'class_2': class_2_list
    })

stratified_df[['actual_response', 'actual_context_ids', 'actual_context', 'class_1', 'class_2']] = stratified_df.progress_apply(process_row, axis=1)


100%|██████████| 125/125 [08:15<00:00,  3.96s/it]


In [82]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,"В контексте нет информации о том, что монетиза...","[170, 1, 182, 140, 27]",Вопрос: query: Что такое Монетизация на RUTUBE...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, ТРАНСЛ...","[Отключение/подключение монетизации, Отключени..."
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Вопрос: query: Подскажите, пожалуйста, что не ...","[0, 170, 182, 194, 119]",Вопрос: query: Что нельзя публиковать на RUTUB...,"[МОДЕРАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОДЕРАЦИ...","[Отклонение/блокировка видео, Отключение/подкл..."
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...",Авторские права на контент принадлежат тому по...,"[7, 5, 6, 224, 4]",Вопрос: query: Кому принадлежат авторские прав...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОНЕТИЗАЦИЯ, ОТСУТСТВУЕ...","[Отклонение/блокировка видео, Отклонение/блоки..."
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,"Вопрос: query: Пройдет видео модерация, если ц...","[12, 11, 8, 195, 9]",Вопрос: query: Можно ли публиковать видео с чт...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки..."
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,Я не знаю ответа на ваш вопрос о произведении ...,"[5, 8, 7, 10, 233]",Вопрос: query: Какие материалы защищены авторс...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, Т...","[Отклонение/блокировка видео, Отклонение/блоки..."
...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,Вопрос: query: Какое видео могу загрузить?\nОт...,"[272, 276, 253, 37, 51]",Вопрос: query: Можно ли скачать видео с RUTUBE...,"[ОТСУТСТВУЕТ, ВИДЕО, ОТСУТСТВУЕТ, ВИДЕО, ПРЕДЛ...","[Отсутствует, Перенос видео с YouTube, Отсутст..."
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,"По контексту нет информации о том, почему нель...","[0, 115, 272, 130, 276]",Вопрос: query: Что нельзя публиковать на RUTUB...,"[МОДЕРАЦИЯ, ПРЕДЛОЖЕНИЯ, ОТСУТСТВУЕТ, ТРАНСЛЯЦ...","[Отклонение/блокировка видео, Плеер, Отсутству..."
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,Какую категорию выбрать для видео?,Категория должна соответствовать содержанию ви...,"[245, 157, 246, 283, 48]",Вопрос: query: Какую категорию выбрать для вид...,"[ВИДЕО, УПРАВЛЕНИЕ АККАУНТОМ, МОДЕРАЦИЯ, МОДЕР...","[Загрузка видео, Персонализация, Смена категор..."
696,У вас встроенные субтитры или сгенерированые?,"Это могут быть субтитры, которые добавил сам а...",Какие субтитры используются в RUTUBE?,283.0,Какие субтитры используются в RUTUBE?,На RUTUBE используются как встроенные субтитры...,"[247, 251, 252, 249, 248]",Вопрос: query: Как включить субтитры в видео?\...,"[ВИДЕО, ПРЕДЛОЖЕНИЯ, ПРЕДЛОЖЕНИЯ, ВИДЕО, ОТСУТ...","[Управление плеером, Плеер, Плеер, Загрузка ви..."


## Retrieve recall

In [83]:
stratified_df['hit_list'] = stratified_df.apply(
    lambda x: [int(int(y) == int(x['index'])) for y in x['actual_context_ids']],
    axis=1
)


In [84]:
stratified_df['recall'] = stratified_df.apply(
    lambda x: sum(x['hit_list']) / 1 if len(x['actual_context_ids']) > 0 else 0,
    axis=1
) # WATCHOUT: везде len(true_context) == 1


In [85]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2,hit_list,recall
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,"В контексте нет информации о том, что монетиза...","[170, 1, 182, 140, 27]",Вопрос: query: Что такое Монетизация на RUTUBE...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, ТРАНСЛ...","[Отключение/подключение монетизации, Отключени...","[0, 1, 0, 0, 0]",1.0
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Вопрос: query: Подскажите, пожалуйста, что не ...","[0, 170, 182, 194, 119]",Вопрос: query: Что нельзя публиковать на RUTUB...,"[МОДЕРАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОДЕРАЦИ...","[Отклонение/блокировка видео, Отключение/подкл...","[0, 0, 0, 0, 0]",0.0
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...",Авторские права на контент принадлежат тому по...,"[7, 5, 6, 224, 4]",Вопрос: query: Кому принадлежат авторские прав...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОНЕТИЗАЦИЯ, ОТСУТСТВУЕ...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0]",1.0
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,"Вопрос: query: Пройдет видео модерация, если ц...","[12, 11, 8, 195, 9]",Вопрос: query: Можно ли публиковать видео с чт...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки...","[0, 0, 1, 0, 0]",1.0
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,Я не знаю ответа на ваш вопрос о произведении ...,"[5, 8, 7, 10, 233]",Вопрос: query: Какие материалы защищены авторс...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, Т...","[Отклонение/блокировка видео, Отклонение/блоки...","[0, 0, 0, 0, 0]",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,Вопрос: query: Какое видео могу загрузить?\nОт...,"[272, 276, 253, 37, 51]",Вопрос: query: Можно ли скачать видео с RUTUBE...,"[ОТСУТСТВУЕТ, ВИДЕО, ОТСУТСТВУЕТ, ВИДЕО, ПРЕДЛ...","[Отсутствует, Перенос видео с YouTube, Отсутст...","[1, 0, 0, 0, 0]",1.0
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,"По контексту нет информации о том, почему нель...","[0, 115, 272, 130, 276]",Вопрос: query: Что нельзя публиковать на RUTUB...,"[МОДЕРАЦИЯ, ПРЕДЛОЖЕНИЯ, ОТСУТСТВУЕТ, ТРАНСЛЯЦ...","[Отклонение/блокировка видео, Плеер, Отсутству...","[0, 0, 0, 0, 0]",0.0
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,Какую категорию выбрать для видео?,Категория должна соответствовать содержанию ви...,"[245, 157, 246, 283, 48]",Вопрос: query: Какую категорию выбрать для вид...,"[ВИДЕО, УПРАВЛЕНИЕ АККАУНТОМ, МОДЕРАЦИЯ, МОДЕР...","[Загрузка видео, Персонализация, Смена категор...","[0, 0, 0, 0, 0]",0.0
696,У вас встроенные субтитры или сгенерированые?,"Это могут быть субтитры, которые добавил сам а...",Какие субтитры используются в RUTUBE?,283.0,Какие субтитры используются в RUTUBE?,На RUTUBE используются как встроенные суб

In [86]:
stratified_df['kb_query_orig'].unique()

array(['Почему могут отключить монетизацию из-за авторского права на видео и как этого избежать?',
       'Какой контент можно использовать для монетизации, то есть в коммерческих целях?',
       'Кому принадлежат авторские права на контент, который я размещаю на платформе?',
       'Можно ли цитировать чужие произведения?',
       'Нарушает ли авторские права использование образа персонажа игры/фильма/сериала?',
       'Что такое мобильное приложение Студия RUTUBE?',
       'Какие функции доступны в приложении Студия RUTUBE?',
       'На каких операционных системах доступно приложение Студия RUTUBE?',
       'Как узнать версию приложения Студия RUTUBE?',
       'Как зарегистрироваться в приложении Студия RUTUBE?',
       'Как авторизоваться в приложении Студия RUTUBE?',
       'Можно ли авторизоваться в приложении Студия RUTUBE под уже существующим аккаунтом?',
       'Можно ли авторизоваться (выполнить вход) в приложении Студия RUTUBE без ввода проверочного кода?',
       'Как выйти 

In [87]:
stratified_df['recall'].mean()

0.208

## Retrieve NDCG

In [88]:
stratified_df['hit_list']

,hit_list
2,"[0, 1, 0, 0, 0]"
9,"[0, 0, 0, 0, 0]"
18,"[1, 0, 0, 0, 0]"
19,"[0, 0, 1, 0, 0]"
32,"[0, 0, 0, 0, 0]"
...,...
733,"[1, 0, 0, 0, 0]"
684,"[0, 0, 0, 0, 0]"
690,"[0, 0, 0, 0, 0]"
696,"[0, 0, 0, 0, 0]"


In [89]:
stratified_df['ndcg'] = stratified_df['hit_list'].apply(lambda x: calculate_ndcg([x])[0]) # КОСТЫЛЬ

In [90]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2,hit_list,recall,ndcg
2,"Все пишут, что монетизация на рутубе отключает...","Добрый день! \nМонетизация может отключиться, ...",Почему могут отключить монетизацию из-за автор...,1.0,Почему могут отключить монетизацию из-за автор...,"В контексте нет информации о том, что монетиза...","[170, 1, 182, 140, 27]",Вопрос: query: Что такое Монетизация на RUTUBE...,"[МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, ТРАНСЛ...","[Отключение/подключение монетизации, Отключени...","[0, 1, 0, 0, 0]",1.0,0.63093
9,"Подскажите, пожалуйста, что не запрещено выкла...","Добрый день! Не запрешено публиковать то, что ...",Какой контент можно использовать для монетизац...,4.0,Какой контент можно использовать для монетизац...,"Вопрос: query: Подскажите, пожалуйста, что не ...","[0, 170, 182, 194, 119]",Вопрос: query: Что нельзя публиковать на RUTUB...,"[МОДЕРАЦИЯ, МОНЕТИЗАЦИЯ, МОНЕТИЗАЦИЯ, МОДЕРАЦИ...","[Отклонение/блокировка видео, Отключение/подкл...","[0, 0, 0, 0, 0]",0.0,0.00000
18,Кому принадлежат авторские права на данный кон...,"По умолчанию считается, что авторские права на...","Кому принадлежат авторские права на контент, к...",7.0,"Кому принадлежат авторские права на контент, к...",Авторские права на контент принадлежат тому по...,"[7, 5, 6, 224, 4]",Вопрос: query: Кому принадлежат авторские прав...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОНЕТИЗАЦИЯ, ОТСУТСТВУЕ...","[Отклонение/блокировка видео, Отклонение/блоки...","[1, 0, 0, 0, 0]",1.0,1.00000
19,"пройдет видео модерация, если цитировать чужие...",Для прохождения модерации нужно соблюдать неск...,Можно ли цитировать чужие произведения?,8.0,Можно ли цитировать чужие произведения?,"Вопрос: query: Пройдет видео модерация, если ц...","[12, 11, 8, 195, 9]",Вопрос: query: Можно ли публиковать видео с чт...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, М...","[Отклонение/блокировка видео, Отклонение/блоки...","[0, 0, 1, 0, 0]",1.0,0.50000
32,произведние в общественном достоянии? О чем это?,"Это значит, что произведение или его перевод п...",Нарушает ли авторские права использование обра...,13.0,Нарушает ли авторские права использование обра...,Я не знаю ответа на ваш вопрос о произведении ...,"[5, 8, 7, 10, 233]",Вопрос: query: Какие материалы защищены авторс...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, Т...","[Отклонение/блокировка видео, Отклонение/блоки...","[0, 0, 0, 0, 0]",0.0,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,Какое видео могу загрузить?,Оно должно соответствовать правилам размещения...,Какие технические требования предъявляются к з...,272.0,Какие технические требования предъявляются к з...,Вопрос: query: Какое видео могу загрузить?\nОт...,"[272, 276, 253, 37, 51]",Вопрос: query: Можно ли скачать видео с RUTUBE...,"[ОТСУТСТВУЕТ, ВИДЕО, ОТСУТСТВУЕТ, ВИДЕО, ПРЕДЛ...","[Отсутствует, Перенос видео с YouTube, Отсутст...","[1, 0, 0, 0, 0]",1.0,1.00000
684,Почнм нельзя смотреть видео в хорошем качестве?,"Вероятнее всего, у вас стоит блокировщик рекла...",Не могу посмотреть ролик в хорошем качестве,277.0,Не могу посмотреть ролик в хорошем качестве,"По контексту нет информации о том, почему нель...","[0, 115, 272, 130, 276]",Вопрос: query: Что нельзя публиковать на RUTUB...,"[МОДЕРАЦИЯ, ПРЕДЛОЖЕНИЯ, ОТСУТСТВУЕТ, ТРАНСЛЯЦ...","[Отклонение/блокировка видео, Плеер, Отсутству...","[0, 0, 0, 0, 0]",0.0,0.00000
690,Как правильно выбрать категорию?,На RUTUBE доступно большое количество категори...,Какую категорию выбрать для видео?,280.0,Какую категорию выбрать для видео?,Категория должна соответствовать содержанию ви...,"[245, 157, 246, 283, 48]",Вопрос: query: Какую категорию выбрать для вид...,"[ВИДЕО, УПРАВЛЕНИЕ АККАУНТОМ, МОДЕРАЦИЯ, МОДЕР...","[Загрузка видео, Персонализация, Смена категор...","[0, 0, 0, 0, 0]",0.0,0.00000
696,У вас встроенные субтитры или сгенерированые?,"Это могут быть субтитры, которые добавил сам а...",Какие субтитры используются в RUTUBE?,283.0,Какие 

In [91]:
stratified_df['ndcg'].mean()

0.17717545310060429

## Generation G-Eval

In [43]:
def clean_and_split_context(text):
    pattern = r"(Вопрос:.*?Ответ:.*?)(?=\nВопрос:|$)"

    matches = re.findall(pattern, text, re.DOTALL)

    result = [match.strip() for match in matches]
    return result


In [46]:
for idx, row in stratified_df.iterrows():
    # display(row)
    # display(row['actual_context'])
    # break



    test_case = LLMTestCase(input=row['input'],
                            actual_output=row['actual_response'],
                            retrieval_context=clean_and_split_context(row['actual_context']))

    relevance_metric.measure(test_case)
    relevance_score = relevance_metric.score
    relevance_reason = relevance_metric.reason

    answer_relevance_metric.measure(test_case)
    answer_relevance_score = answer_relevance_metric.score
    answer_relevance_reason = answer_relevance_metric.reason

    hallucination_detection_metric.measure(test_case)
    hallucination_detection_score = hallucination_detection_metric.score
    hallucination_detection_reason = hallucination_detection_metric.reason

    stratified_df.at[idx, 'relevance_score'] = relevance_score
    stratified_df.at[idx, 'relevance_reason'] = relevance_reason

    stratified_df.at[idx, 'answer_relevance_score'] = answer_relevance_score
    stratified_df.at[idx, 'answer_relevance_reason'] = answer_relevance_reason

    stratified_df.at[idx, 'hallucination_detection_score'] = hallucination_detection_score
    stratified_df.at[idx, 'hallucination_detection_reason'] = hallucination_detection_reason


Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

...

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

Output()

Event loop is already running. Applying nest_asyncio patch to allow async execution...

In [47]:
stratified_df

,input,expected_output,kb_query,index,kb_query_orig,actual_response,actual_context_ids,actual_context,class_1,class_2,hit_list,recall,ndcg,relevance_score,relevance_reason,answer_relevance_score,answer_relevance_reason,hallucination_detection_score,hallucination_detection_reason
181,как с ютуба к вам импортировать контент? В при...,Перенести видео с YouTube на RUTUBE можно толь...,Как перенести видео с YouTube на RUTUBE?,74.0,Как перенести видео с YouTube на RUTUBE?,Перенести видео с YouTube на Rutube можно толь...,"[42, 51, 277, 276, 47]",Вопрос: query: Как перенести видео c YouTube (...,"[ПРЕДЛОЖЕНИЯ, ПРЕДЛОЖЕНИЯ, ВИДЕО, ВИДЕО, ВИДЕО]","[Студия RUTUBE, Студия RUTUBE, Перенос видео с...","[0, 0, 0, 0, 0]",0.0,0.0,0.0,The retrieval context provides relevant inform...,0.0,The actual output partially addresses the main...,1.0,All information in the actual output is direct...
223,"значю только редактор есть и модератор, а каки...","Всего существуют пять ролей, каждая роль огран...",Какие бывают роли в доступах к студии канала?,92.0,Какие бывают роли в доступах к студии канала?,Основные роли в RUTUBE для каналов — это Владе...,"[64, 58, 73, 72, 63]","Вопрос: query: Может ли пользователь, которому...","[ПРЕДЛОЖЕНИЯ, УПРАВЛЕНИЕ АККАУНТОМ, ПРЕДЛОЖЕНИ...","[Студия RUTUBE, Персонализация, Студия RUTUBE,...","[0, 0, 0, 0, 0]",0.0,0.0,0.0,The retrieval context discusses roles and acce...,0.0,The response addresses the main point about ch...,0.0,The actual output introduces the role of 'Помо...
436,Какие параметры можно поменять у встроенного п...,После вставки кода видео в код страницы в него...,Какие настройки параметров можно сделать в код...,178.0,Какие настройки параметров можно сделать в код...,У встроенного плеера можно поменять параметры:...,"[155, 254, 266, 25, 96]",Вопрос: query: Какие настройки параметров можн...,"[ВИДЕО, ВИДЕО, ВИДЕО, УПРАВЛЕНИЕ АККАУНТОМ, ВИ...","[Встраивание видео, Загрузка видео, Загрузка в...","[0, 0, 0, 0, 0]",0.0,0.0,1.0,The retrieval context provides detailed parame...,1.0,The actual output directly addresses the input...,1.0,All parameters mentioned in the actual output ...
562,Кто проводит модерацию?,Модерация бывает ручной (проверка модератором-...,Кем осуществляется модерация в RUTUBE?,228.0,Кем осуществляется модерация в RUTUBE?,Модерация в RUTUBE осуществляется модераторами...,"[195, 197, 196, 194, 77]",Вопрос: query: Что проверяют при модерации?\nО...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, У...","[Отклонение/блокировка видео, Долгая модерация...","[0, 0, 0, 0, 0]",0.0,0.0,1.0,The retrieval context includes relevant inform...,1.0,The actual output directly addresses the input...,1.0,The actual output accurately reflects the retr...
440,"Я пока еще новичок, только зарегистрировался н...",Настройки вашего канала доступны по ссылке: ht...,Как мне лучше оформить свой канал?,180.0,Как мне лучше оформить свой канал?,Для того чтобы лучше оформить свой канал на Ru...,"[157, 215, 216, 201, 243]",Вопрос: query: Как мне лучше оформить свой кан...,"[УПРАВЛЕНИЕ АККАУНТОМ, УПРАВЛЕНИЕ АККАУНТОМ, У...","[Персонализация, Регистрация/Авторизация, Перс...","[0, 0, 0, 0, 0]",0.0,0.0,1.0,The retrieval context provides detailed steps ...,0.0,The actual output directly addresses the main ...,1.0,The actual output accurately reflects the info...
567,"Здравствуйте! При модерации высветилось ""Видео...","Пришлите, пожалуйста, ссылку на видео. Уточним...",Что значит «Ваше видео отклонено»?,230.0,Что значит «Ваше видео отклонено»?,"Здравствуйте! К сожалению, я не могу знать при...","[271, 260, 255, 244, 239]",Вопрос: query: На моем видео стоит значок «18+...,"[МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, МОДЕРАЦИЯ, В...","[Смена категории/возрастные ограничения, Запре...","[0, 0, 0, 0, 0]",0.0,0.0,0.0,The retrieval context does not directly addres...,0.0,The response acknowledges the user's issue but...,0.0,The actual output suggests contacting support ...
322,У нас дома два телевизора на кухне и в комнате...,"Конечно, вы можете 